# Парсинг DICE.COM

Никита Метелёв

## Подготовка среды, импорт библиотек

Инсталлируем необходимые пакеты и обеспечим возможность работы Selenium в Colab

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

apt-get update
apt-get install chromium chromium-driver
pip3 install selenium

Executing: /tmp/apt-key-gpghome.uCc932u0c0/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.Ar2aADU9yw/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.jNk5PHksdq/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

Импортируем необходимые библиотеки:

In [2]:
import time
import csv

from datetime import date

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

import requests
import pandas as pd
from bs4 import BeautifulSoup

import warnings
warnings.simplefilter("ignore", ResourceWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.filterwarnings('ignore')

## Инициализируем переменные

In [3]:
headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "X-Is-Ajax-Request": "X-Is-Ajax-Request",
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": ("Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36")
    }

Задаём параметры для корректной работы Selenium

In [4]:
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
service = Service("chromedriver")

Функция определяет __дату публикации__ исходя из прошедшего времени относительно московского
работает только для вакансий за последние 24 часа

На вход получает строку типа:
- _"Posted 2 hours ago"_
- _"Posted moments ago"_

Возвращает дату публикации вакансии

In [5]:
def dt_job(time_ago):
  if time_ago.split()[1] == 'moments':
    hours = -3
  else:
    hours = int(time_ago.split()[1]) - 3
  date = (pd.datetime.now() - pd.DateOffset(hours=hours)).date()
  return date

Создаем пустой список для ссылок на вакансии

In [6]:
list_href = []

Создаем первый датафрейм, куда внесем информацию со страницы поиска

Создаем список для подробной информации по каждой вакансии

In [7]:
job_data = []

Ссылка на страницу поиска _dice.com_ с запросом:
- junior data analyst OR scientist
- только за последние 24 часа
- 300 вакансий на странице

Больше 150 вакансий за день не видел, обычно около 50.
Листать не придется

In [8]:
vpp = 300 # количество вакансий на одной странице
dice_url = ('https://www.dice.com/jobs?q=junior%20'
            'data%20analyst%20OR%20scientist&countryCode=US'
            f'&radius=30&radiusUnit=mi&page=1&pageSize={vpp}&filters.'
            'postedDate=ONE&language=en&eid=S2Q_')

## Рабочий цикл

С Selenium открываем страницу поиска

In [9]:
driver = webdriver.Chrome(service=service, options=options)
driver.get(dice_url)
driver.maximize_window()
time.sleep(7)

Собираем ссылки на вакансии

In [10]:
for elem in driver.find_elements(By.XPATH,'//*[@class="card-title-link bold"]'):
    list_href.append(elem.get_attribute('href'))

По каждой вакансии достаем информацию со страницы поиска

In [11]:
df = []
for i in range(1, len(list_href)+1):

  job_post = (driver.
              find_element
              (By.XPATH,
               '//*[@id="searchDisplay-div"]/div[3]/dhi-search-'
              f'cards-widget/div/dhi-search-card[{i}]').
              text.splitlines()[0])

  comp_loc = (driver.
              find_element
              (By.XPATH,
                '//*[@id="searchDisplay-div"]/div[3]/dhi-search-'
              f'cards-widget/div/dhi-search-card[{i}]'
                '/div/div[1]/div/div[2]/div[1]/div').
              text.splitlines())

  company = comp_loc[0]

  if len(comp_loc) > 1:
     location = comp_loc[1].removesuffix(', USA')
  else:
    location = None

  working_conditions = (driver.
                        find_element
                        (By.XPATH,
                         '//*[@id="searchDisplay-div"]/div[3]'
                        f'/dhi-search-cards-widget/div/dhi-search-card[{i}]'
                         '/div/div[2]/div[1]').
                        text.splitlines()[0])

  posted = (driver.
            find_element
            (By.XPATH, '//*[@id="searchDisplay-div"]/div[3]/dhi-search'
            f'-cards-widget/div/dhi-search-card[{i}]/div/div[2]/div[1]').
            text.splitlines()[1])

  info_another = (driver.find_element
                     (By.XPATH, '//*[@id="searchDisplay-div"]/div[3]/dhi-'
                     f'search-cards-widget/div/dhi-search-card[{i}]'
                     '/div/div[2]/div[2]')
                     .text.splitlines()[0])

  df.append({'title' : job_post,
            'company' : company,
            'country' : 'USA',
            'location' : location,
            'date' : dt_job(posted),
            'company_field' : None,
            'job_type' : working_conditions,
            'info_another' : info_another})
driver.close()
driver.quit()

In [12]:
df

[{'title': 'Junior Data Analyst',
  'company': 'Motion Recruitment',
  'country': 'USA',
  'location': 'Chesterfield, MO',
  'date': datetime.date(2023, 6, 21),
  'company_field': None,
  'job_type': 'Full-time',
  'info_another': 'An innovative agriculture technology startup is seeking a talented Data Analyst to contribute to their mission of harnessing advanced technology and expertise to unlock the potential of agricultural innovations. Come join a fully remote team where you will help revolutionize the agriculture industry through data-driven insights and cutting-edge technology! You will play a crucial role in collecting and interpreting geospatial agronomic data with precision and speed, generating actionable insight'},
 {'title': 'Junior Data Scientist',
  'company': 'HTC Global Services',
  'country': 'USA',
  'location': 'Dearborn, MI',
  'date': datetime.date(2023, 6, 21),
  'company_field': None,
  'job_type': 'Full-time',
  'info_another': "HTC Global Services wants you. Co

Заходим в каждую вакансию и собираем:
- полное описание,
- хард скилы,
- компенсацию

In [13]:
for url in list_href:
    time.sleep(1)
    # некоторые ссылки ведут на сторонние сайты, поэтому проверяем домен
    # прежде чем продолжить
    if url[:20] == 'https://www.dice.com':
      req = requests.get(url=url, headers=headers)
      soup = BeautifulSoup(req.text, "lxml")

      skills = ''
      for skill in soup.find_all(class_="skill-badge"):
          skills += skill.text + ', '
      skills = skills.strip(', ')

      dscr= soup.find('div', class_="mb-16 min-h-[300px]").text

      # зарплата указана не во всех вакансиях,
      # сначала проверяю наличие информации
      salary = None
      salary_div = soup.find('div',
                        class_='job-info order-4 col-span-2 mb-10 md:mb-0 '
                        'sm:col-span-1 md:col-span-4 lg:col-span-5 lg:mb-0')

      for element in salary_div:
        if element.get('data-cy') == 'compensationText':
          salary = (soup.find('div',
                        class_='job-info order-4 col-span-2 mb-10 md:mb-0 sm:'
                        'col-span-1 md:col-span-4 lg:col-span-5 lg:mb-0').
                        find('p').
                        text)
    else:
      skills = None
      dscr = '-'
      salary = None
    data = {
        'salary' : salary,
        'source' : 'dice.com',
        'link' : url,
        'description' : dscr,
        'skills' : skills
    }
    job_data.append(data)
    print("Saving")

Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving
Saving


Объединяем результаты и выгружаем в CSV

In [14]:
with open(f'dice_jobs {date.today()}', mode="w", encoding='utf-8') as w_file:
    names = ['title',
           'company',
           'country',
           'location',
           'salary',
           'source',
           'link',
           'date',
           'company_field',
           'description',
           'skills',
           'job_type']
    file_writer = csv.DictWriter(w_file, delimiter = ",",
                                 lineterminator="\r", fieldnames=names)
    file_writer.writeheader()
    for i in range(len(list_href)):
      row = job_data[i] | df[i]
      if row['description'] == '-':
        row['description'] = row['info_another']
      del row['info_another']
      file_writer.writerow(row)

В итоге получили csv файл с вакансиями за последные сутки с сайта dice.com для новичков в Data Science